In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

## 策略模型定义

In [2]:
import torch
import torch.nn as nn

class LSTMPolicyNet(nn.Module):
    def __init__(self, state_dim=10, lstm_hidden=128, mlp_hidden=128, num_layers=1, dropout=0.1):
        super().__init__()
        self.state_dim = state_dim
        self.lstm_hidden = lstm_hidden
        self.num_layers = num_layers

        self.lstm = nn.LSTM(
            input_size=state_dim,
            hidden_size=lstm_hidden,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        self.mlp = nn.Sequential(
            nn.Linear(lstm_hidden + state_dim, mlp_hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_hidden, 2)  # 输出2个动作的logits
        )

    def forward(self, state_seq, hidden=None):
        """
        state_seq: [B, T, state_dim]  # batch, seq_len, state_dim
        hidden: (h0, c0) tuple for LSTM initial state (optional)
        Returns:
            action_logits: [B, T, 2]  # 每个时刻的动作分数
        """
        lstm_out, hidden = self.lstm(state_seq, hidden)  # lstm_out: [B, T, lstm_hidden]
        logits = self.mlp(torch.cat((lstm_out, state_seq), dim=-1))  # [B, T, lstm_hidden + state_dim] -> [B, T, 2]
        return logits, hidden

    def act(self, state_seq, hidden=None, deterministic=False):
        """
        用于采样动作
        state_seq: [B, T, state_dim]
        Returns:
            actions: [B, T]
        """
        logits, hidden = self.forward(state_seq, hidden)
        probs = torch.softmax(logits, dim=-1)
        if deterministic:
            actions = torch.argmax(probs, dim=-1)
        else:
            dist = torch.distributions.Categorical(probs)
            actions = dist.sample()
        return actions, probs, hidden

    def reset_hidden(self, batch_size):
        """
        重置LSTM的隐藏状态
        batch_size: int
        Returns:
            hidden: (h0, c0) tuple for LSTM initial state
        """
        h0 = torch.zeros(self.num_layers, batch_size, self.lstm_hidden).to(next(self.parameters()).device)
        c0 = torch.zeros(self.num_layers, batch_size, self.lstm_hidden).to(next(self.parameters()).device)
        return (h0, c0)

## 加载ReplayDataset

In [3]:
import datasets

ReplayDataset = datasets.load_from_disk("../data/scores_rb/shareGPT-llama3-d7-topk10-t1")

In [4]:
# ReplayDataset.set_format("torch")
# type(ReplayDataset["train"][0]["action_7"]["stop"])

In [5]:
ReplayDataset["train"][0]

{'eagle_1_forward': [-1.028,
  -1.31,
  -1.489,
  -2.748,
  -3.365,
  -3.592,
  -4.94,
  -5.656,
  -5.742,
  -6.645],
 'eagle_2_forward': [-1.049,
  -1.315,
  -1.489,
  -3.074,
  -3.375,
  -3.592,
  -4.56,
  -4.926,
  -4.996,
  -5.34],
 'eagle_3_forward': [-1.052,
  -1.491,
  -2.14,
  -2.953,
  -2.953,
  -3.14,
  -3.879,
  -4.074,
  -4.273,
  -4.555],
 'eagle_4_forward': [-1.326,
  -1.562,
  -2.955,
  -3.25,
  -3.305,
  -3.719,
  -3.84,
  -3.922,
  -4.133,
  -4.258],
 'eagle_5_forward': [-1.343,
  -1.564,
  -3.41,
  -3.58,
  -3.846,
  -3.986,
  -4.27,
  -4.465,
  -5.023,
  -5.06],
 'eagle_6_forward': [-1.343,
  -1.565,
  -3.426,
  -3.852,
  -4.664,
  -4.68,
  -4.734,
  -4.85,
  -5.36,
  -5.51],
 'eagle_7_forward': [-1.72,
  -1.849,
  -2.86,
  -3.098,
  -3.719,
  -4.453,
  -4.453,
  -4.516,
  -4.684,
  -5.055],
 'eagle_8_forward': [-2.912,
  -2.938,
  -3.562,
  -3.7,
  -3.781,
  -3.9,
  -3.93,
  -3.945,
  -4.055,
  -4.08],
 'action_0': {'stop': [0.00017976760864257812, 0.999776840209960

## 奖励函数

In [6]:
import numpy as np
# 1. 奖励函数（期望reward）
def reward_fn(action_idx, stop_probs, lam1=1.0, lam2=0.9):
    n = action_idx
    probs = np.array(stop_probs[1:], dtype=np.float32)
    probs[0] += stop_probs[0]  # 合并长度为0和1的概率
    l_arr = np.arange(len(probs))  # [0, 1, ..., len(probs)-1]
    reward_arr = lam1 * l_arr - lam2 * np.abs(n - l_arr)
    expected_reward = np.sum(probs * reward_arr)
    return expected_reward

In [7]:
def avg_acc_len_fn(stop_probs):
    return np.sum(np.arange(len(stop_probs)) * np.array(stop_probs, dtype=np.float32))

def avg_acc_len_fn_tensor(stop_probs:torch.Tensor):
    """stop_probs: [B, T]"""
    return torch.sum(torch.arange(stop_probs.size(-1), device=stop_probs.device) * stop_probs, dim=-1)

def padding_stops(stops, max_len=9):
    """
    对stops进行padding，使其长度为max_len
    0~8共9个位置
    """
    padded_stops = stops + [0.0] * (max_len - len(stops))
    return padded_stops[:max_len]  # 确保长度不超过max_len

## 数据集定义

In [8]:
from torch.utils.data import Dataset
import numpy as np

def normalize_probs(arr):
    arr = np.clip(arr, 0, None)
    row_sums = arr.sum(axis=-1, keepdims=True)
    # 如果存在全为0的行就报错
    if np.any(row_sums <= 1e-8):
        raise ValueError("dataset error")
    arr = arr / row_sums
    return arr

class SharedStatesDataset(Dataset):
    def __init__(self, dataset, max_len=7):
        self.samples = []
        for sample in dataset:
            # 仅用eagle_1_forward ~ eagle_7_forward
            state_seq = [sample[f'eagle_{i}_forward'] for i in range(1, max_len+1)]  # [7, state_dim]
            stops = [padding_stops(sample[f'action_{i}']['stop']) for i in range(max_len+1)]
            stops = [normalize_probs(np.array(stop)) for stop in stops]
            # rewards = [reward_fn(i, stops[i]) for i in range(max_len+1)]
            avg_acc_lens_op = avg_acc_len_fn(stops[-1])  # 最优平均接受长度
            self.samples.append({
                "states": state_seq,     # [7, state_dim]
                # "all_rewards": rewards,  # [8]
                "stop_probs": stops,
                "avg_acc_lens_op": avg_acc_lens_op
            })

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

def collate_fn(batch):
    states = torch.tensor([x["states"] for x in batch], dtype=torch.float32) # [B, 7, state_dim]
    # all_rewards = torch.tensor([x["all_rewards"] for x in batch], dtype=torch.float32) # [B, 8]
    stop_probs = torch.tensor([x["stop_probs"] for x in batch], dtype=torch.float32)  # [B, 8, 9]
    avg_acc_lens_op = torch.tensor([x["avg_acc_lens_op"] for x in batch], dtype=torch.float32)  # [B]
    return states, stop_probs, avg_acc_lens_op  
    # return states, all_rewards


## 参数

In [9]:
max_len = 7
num_paths = max_len + 1  # 8

## 构造奖励矩阵

In [10]:
ACTION_SEQ_MATRIX = torch.stack([
        torch.tensor([1] * i + [0] * (num_paths - i - 1), dtype=torch.long)
        for i in range(num_paths)
    ])  # [8, 7]
MASK_MATRIX = torch.tril(torch.ones((num_paths, num_paths), dtype=torch.float32))[:, :-1]  # [8, 7]

In [11]:
ACTION_SEQ_MATRIX

tensor([[0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1]])

In [12]:
MASK_MATRIX

tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1.]])

In [13]:
from torch.utils.data import DataLoader
train_set = ReplayDataset["train"]
test_set = ReplayDataset["test"]
train_dataset = SharedStatesDataset(train_set, max_len=max_len)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)
test_dataset = SharedStatesDataset(test_set, max_len=max_len)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn)

In [14]:
def make_reward_matrix(rate=1, maxlen=7):
    rewards = rate * torch.ones(maxlen+2, maxlen+1, maxlen+1, dtype=torch.long)
    for al in range(9):
        rewards[al, :, max(al-1, 0):] = -1
    rewards = rewards[:,:,:maxlen]

    action = torch.tensor(
        [
            [1] * i + [0] * (maxlen - i) for i in range(maxlen + 1)
        ]
    )

    mask = torch.tril(torch.ones(maxlen+1,maxlen+1,dtype=torch.long))[:,:maxlen]

    R = rewards * (2 * action - 1) * mask # [9,8,7] acclen,action_idx,reward
    return R

def make_g_matrix(reward_matrix, maxlen=7, gamma=0.9):
    G = torch.zeros_like(reward_matrix, dtype=torch.float32)
    G[:,:,maxlen-1] = reward_matrix[:,:,maxlen-1]  # 最后一个时间步的G值等于奖励
    for i in reversed(range(1, maxlen)):
        G[:,:,i-1] = reward_matrix[:,:,i-1] + gamma * G[:,:,i]
    return G # [9,8,7] acclen,action_idx,gamma_reward

## 统计接受长度分布、平均接受长度

In [15]:
def mask_after_first_zero(tensor):
    """
    参数:
        tensor: 输入张量，形状为[bsz, 8],仅包含整数值0或1
    
    返回:
        形状为[bsz, 8]的张量,第一个0之后的所有位置被置0
    """
    # 1. 找出所有0的位置
    zeros = tensor.eq(0)
    
    # 2. 创建累积mask（为第一个0之后的位置标记为False）
    # 使用cumprod将第一个0之前的True保留，之后的变为False
    mask = torch.cat(
        [torch.ones(zeros.size(0), 1, dtype=torch.bool, device=tensor.device),
         ~zeros[:, :-1]], 
        dim=1
    ).cumprod(dim=1).bool()
    
    return tensor * mask

def find_first_zero_pos(tensor):
    """
    参数:
        tensor: 输入张量，形状为[bsz, 8]，仅包含整数值0或1
    
    返回:
        形状为[bsz, 1]的张量，表示每行第一个0的位置（若没有则为8）
    """
    # 步骤1: 创建布尔掩码标记0的位置
    mask = tensor.eq(0)  # 等同于 (tensor == 0)
    mask = mask.to(torch.long)
    
    # 步骤2: 在最后一维末尾添加全True的列
    extended_mask = torch.cat(
        [mask, torch.ones(mask.size(0), 1, dtype=torch.long, device=tensor.device)],
        dim=1
    )
    
    # 步骤3: 沿最后一维查找第一个True的位置索引
    first_zero_pos = extended_mask.argmax(dim=1, keepdim=True)
    return first_zero_pos

from collections import defaultdict
def cal_avg_len(model, data_loader):
    model.eval()
    len_dict = defaultdict(list)

    with torch.no_grad():
        for states, stop_probs, avg_acc_lens_op in data_loader:
            # states: [B, 7, state_dim], stop_probs: [B, 8, 8], avg_acc_lens_op: [B]
            batch_size = states.size(0)
            hidden = model.reset_hidden(batch_size)
            actions, _, _ = model.act(states, hidden)  # [B, T]
            # actions中第一个0出现的位置为动作序列长度
            action_length = find_first_zero_pos(actions)  # [B, 1]
            expanded_index = action_length.unsqueeze(-1).expand(-1, -1, stop_probs.size(-1))  # [B, 1, max_len + 1]
            action_stop_probs = stop_probs.gather(dim=1, index=expanded_index).squeeze(1) # [B, max_len + 1]
            # 计算平均接受长度
            avg_acc_lens = avg_acc_len_fn_tensor(action_stop_probs) # [B]
            len_dict["avg_acc_lens"].extend(avg_acc_lens.cpu().numpy())
            # 计算与最优平均接受长度差值
            optimal_avg_acc_lens = avg_acc_lens - avg_acc_lens_op
            len_dict["optimal_avg_acc_lens_diff"].extend(optimal_avg_acc_lens.cpu().numpy())
            
    return len_dict
    

## 测试代码

In [ ]:
def evaluate_loss(model, data_loader, device):
    model.eval()
    total_loss = 0
    total_batches = 0
    G = make_g_matrix(make_reward_matrix(rate=2, maxlen=max_len), maxlen=max_len, gamma=0.8)  # [9, 8, 7]
    G = G.to(device)

    with torch.no_grad():
        for states, stop_probs, _ in data_loader:
            states = states.to(device)
            stop_probs = stop_probs.to(device)  # [B, 8, 9]
            logits, _ = model(states)  # [B, 7, 2]

            log_probs = nn.functional.log_softmax(logits, dim=-1)  # [B, 7, 2]
            probs = torch.softmax(logits, dim=-1)  # [B, 7, 2]
            action_dist = torch.distributions.Categorical(probs=probs)  # 创建Categorical分布
            sampled_actions = action_dist.sample()  # 采样动作 [B, 7]
            action_indices = find_first_zero_pos(sampled_actions)  # [B, 1]
            accept_length_dist = torch.distributions.Categorical(
                probs=stop_probs.gather(dim=1, index=action_indices.unsqueeze(-1).expand(-1, -1, stop_probs.size(-1)))
            )
            accept_lengths = accept_length_dist.sample()  # [B, 1]
            # 计算回报 g = G[action_idx, accept_length, :]
            g_matrix = G[accept_lengths.squeeze(-1).to(device), action_indices.squeeze(-1).to(device), :]  # [B, 7]
            # 计算损失
            action_log_probs = log_probs.gather(dim=-1, index=sampled_actions.unsqueeze(-1)).squeeze(-1)  # [B, 7]
            loss = -(action_log_probs * g_matrix).mean()  # 平均损失
            total_loss += loss.item()
            total_batches += 1
    model.train()
    return total_loss / total_batches if total_batches > 0 else float('inf')

## 训练代码

In [17]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt


state_dim = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu") 
model = LSTMPolicyNet(state_dim=state_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-4)  

action_seq_matrix = ACTION_SEQ_MATRIX
mask_matrix = MASK_MATRIX

num_epochs = 100
train_losses = []
test_losses = []
# avg_lens = []
best_test_loss = float('inf')
G = make_g_matrix(make_reward_matrix(rate=2, maxlen=max_len), maxlen=max_len, gamma=0.8)  # [9, 8, 7]
G = G.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    total_batches = 0
    for states, stop_probs, _ in train_loader:
        states = states.to(device)
        stop_probs = stop_probs.to(device)  # [B, 8, 9]

        logits, _ = model(states)  # [B, 7, 2]
        log_probs = nn.functional.log_softmax(logits, dim=-1)  # [B, 7, 2]
        probs = torch.softmax(logits, dim=-1)  # [B, 7, 2]
        action_dist = torch.distributions.Categorical(probs=probs)  # 创建Categorical分布
        sampled_actions = action_dist.sample()  # 采样动作 [B, 7]
        action_indices = find_first_zero_pos(sampled_actions)  # [B, 1]
        accept_length_dist = torch.distributions.Categorical(
            probs=stop_probs.gather(dim=1, index=action_indices.unsqueeze(-1).expand(-1, -1, stop_probs.size(-1)))
        )
        accept_lengths = accept_length_dist.sample()  # [B, 1]
        # 计算回报 g = G[action_idx, accept_length, :]
        g_matrix = G[accept_lengths.squeeze(-1).to(device), action_indices.squeeze(-1).to(device), :]  # [B, 7]
        # 计算损失
        action_log_probs = log_probs.gather(dim=-1, index=sampled_actions.unsqueeze(-1)).squeeze(-1)  # [B, 7]
        loss = -(action_log_probs * g_matrix).mean()  # 平均损失
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        total_loss += loss.item()
        total_batches += 1

    avg_train_loss = total_loss / total_batches if total_batches > 0 else 0
    avg_test_loss = evaluate_loss(model, test_loader, device)
    # avg_len = cal_avg_len(model, test_loader)
    train_losses.append(avg_train_loss)
    test_losses.append(avg_test_loss)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}")

    if avg_test_loss < best_test_loss:
            best_test_loss = avg_test_loss
            torch.save(model.state_dict(), "lstm_policy.pt")
            print(f"Best test loss updated: {best_test_loss:.4f}, model checkpoint saved.")

# 可视化loss
plt.figure()
plt.plot(range(1, num_epochs+1), train_losses, label="Train Loss")
plt.plot(range(1, num_epochs+1), test_losses, label="Test Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.title("Training/Test Loss Curve")
plt.savefig("loss_curve.png")
plt.show()

torch.save(model.state_dict(), "lstm_policy.pt")

/tmp/ipykernel_1545077/2940358126.py:39: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  stop_probs = torch.tensor([x["stop_probs"] for x in batch], dtype=torch.float32)  # [B, 8, 9]


RuntimeError: indices should be either on cpu or on the same device as the indexed tensor (cpu)